In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Zaman Serisi Formatına Getirme
def create_sequences(data, target, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length].flatten())  # MLP için düzleştirilmiş giriş
        targets.append(target[i + seq_length])
    return np.array(sequences), np.array(targets)

SEQ_LENGTH = 10
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_seq, y_seq = create_sequences(X_scaled, y.values, SEQ_LENGTH)

# PCA ve LDA Veri Hazırlığı
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_seq)

lda = LDA(n_components=1)
X_lda = lda.fit_transform(X_seq, y_seq)

# Performans Metriklerini Hesaplama
def calculate_metrics(y_true, y_pred, y_pred_proba):
    return {
        "Accuracy": accuracy_score(y_true, y_pred) * 100,
        "Precision": precision_score(y_true, y_pred) * 100,
        "Recall": recall_score(y_true, y_pred) * 100,
        "F1-Score": f1_score(y_true, y_pred) * 100,
        "AUC": roc_auc_score(y_true, y_pred_proba) * 100
    }

# MLP Modeli Tanımlama
def build_mlp_model(input_shape):
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Eğitim ve Test Performansını Hesaplama
results_train = []
results_test = []

for reduction, X_data in zip(
    ['No PCA/LDA', 'PCA', 'LDA'],
    [X_seq, X_pca, X_lda]
):
    # Eğitim ve Test Bölünmesi
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_seq, test_size=0.2, random_state=42, stratify=y_seq)

    # Modeli Eğit
    mlp_model = build_mlp_model(X_train.shape[1])
    mlp_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

    # Eğitim Kümesi Performansı
    y_train_pred = (mlp_model.predict(X_train) > 0.5).astype("int32").flatten()
    y_train_proba = mlp_model.predict(X_train).flatten()
    train_metrics = calculate_metrics(y_train, y_train_pred, y_train_proba)
    train_metrics['Reduction'] = reduction
    train_metrics['Model'] = 'MLP'
    results_train.append(train_metrics)

    # Test Kümesi Performansı
    y_test_pred = (mlp_model.predict(X_test) > 0.5).astype("int32").flatten()
    y_test_proba = mlp_model.predict(X_test).flatten()
    test_metrics = calculate_metrics(y_test, y_test_pred, y_test_proba)
    test_metrics['Reduction'] = reduction
    test_metrics['Model'] = 'MLP'
    results_test.append(test_metrics)

# Sonuçları Görselleştirme
train_df = pd.DataFrame(results_train)
test_df = pd.DataFrame(results_test)

# Eğitim ve Test Performans Sonuçları Tablosu
print("Eğitim Kümesi Performans Sonuçları:")
print(train_df)
print("\nTest Kümesi Performans Sonuçları:")
print(test_df)

# Sonuçları Kaydetme
train_df.to_csv("mlp_train_performance.csv", index=False)
test_df.to_csv("mlp_test_performance.csv", index=False)
print("\nSonuçlar 'mlp_train_performance.csv' ve 'mlp_test_performance.csv' dosyalarına kaydedildi.")

Epoch 1/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8266 - loss: 0.4148
Epoch 2/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8675 - loss: 0.3235
Epoch 3/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8706 - loss: 0.3084
Epoch 4/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8775 - loss: 0.2984
Epoch 5/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8797 - loss: 0.2898
Epoch 6/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8759 - loss: 0.2905
Epoch 7/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8748 - loss: 0.2912
Epoch 8/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8776 - loss: 0.2856
Epoch 9/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8797 - loss: 0.2807
Epoch 10/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8835 - loss: 0.2755
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 616us/step
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 1s 539us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 